## Boilerplate

Runs llama3 locally, using Ollama

In [1]:
from langchain_community.llms import Ollama

llm = Ollama(
    model = "llama3"
)

In [2]:
llm.invoke("Hi, who are you?")

"Nice to meet you! I'm LLaMA, an AI assistant developed by Meta AI that can understand and respond to human input in a conversational manner. I'm not a human, but a computer program designed to simulate conversation and answer questions to the best of my ability based on my training data. My purpose is to provide helpful and accurate information, entertain with creative ideas, or simply be a friendly ear to listen. What would you like to chat about?"

Parisng to a formated form of output from model, multiple kind of parsers available at https://python.langchain.com/v0.1/docs/modules/model_io/output_parsers/

In [3]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

# forming a chain
chain = llm | parser
chain.invoke("Hi, do you know about Ahmadiyya Community? Who was founder of Ahmadiyya Community?")


"Yes, I'm familiar with the Ahmadiyya Community. The Ahmadiyya Movement in Islam is a religious movement within Islam that was founded by Mirza Ghulam Ahmad (1835-1908) in British India in 1889.\n\nMirza Ghulam Ahmad, who is considered the Messiah and the Promised Reformer by the Ahmadiyya Community, claimed to be the expected Mahdi (guided one) and the Qiamat (resurrection of Islam). He believed that his role was to revive the true teachings of Islam and restore its original purity.\n\nAhmad's movement gained popularity in India during the late 19th century, particularly among Muslims who were disillusioned with the corruption and division within their own community. His followers, known as Ahmadis, saw him as a reformer who would bring peace, prosperity, and unity to the Muslim world.\n\nThe Ahmadiyya Community has since spread globally, with millions of adherents worldwide. They are known for their commitment to interfaith dialogue, social service, and education. The community has a

Introducing Prompts

In [4]:
from langchain.prompts import PromptTemplate

prompt = ( PromptTemplate.from_template(
    "Tell me a fact about {entity}"
)
)

prompt.format(entity="cricket")


'Tell me a fact about cricket'

In [5]:
prompt

PromptTemplate(input_variables=['entity'], template='Tell me a fact about {entity}')

LLM Chains

In [6]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt)
chain.invoke({"entity": "Balls"})

/home/linux/nlp/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


{'entity': 'Balls',
 'text': 'An interesting topic!\n\nHere\'s a fact: Did you know that the world\'s largest ball of twine was created in Cawker, Kansas, USA? It was made by wrapping over 20,000 pounds (around 9,100 kilograms) of twine around a steel drum, and it measures about 12 feet (3.6 meters) tall! The giant ball of twine is now on display at the Monument Rocks, also known as the "Chalk Pyramids," in western Kansas.'}

In [52]:
for s in llm.stream(
    "Can you write a song about rain on a summer day?"
):
    print(s, end="", flush=True)


Here's a song I came up with:

**Rain on a Summer Day**

(Verse 1)
The sun was shining bright, kids playing outside all night
But then the clouds rolled in, and everything felt just right
A gentle patter starts to fall, a sweet and soothing call
Rain on a summer day, it's a feeling that never gets old

**(Chorus)**
Rain on a summer day, a surprise in every way
Bringing life to our parched earth, in a refreshing display
We'll dance beneath the droplets, spinning round in circles free
Letting go of our worries, just us and the rain, wild and carefree

**(Verse 2)**
The scent of wet earth rises, a primal symphony plays
As petals lift to the sky, drinking in the rain's sweet sway
The world awakens anew, refreshed from its summer haze
Rain on a summer day, washing away all our dismay

**(Chorus)**
Rain on a summer day, a surprise in every way
Bringing life to our parched earth, in a refreshing display
We'll dance beneath the droplets, spinning round in circles free
Letting go of our worries

# RAG

In [7]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

# Load PDF documents

loader = PyPDFDirectoryLoader('./data')
documents = loader.load()

In [8]:
documents[0]

Document(page_content='', metadata={'source': 'data/Philosophy-of-Teachings-of-Islam.pdf', 'page': 0})

In [9]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter


# Split documents into manageable chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)

docs[:4]

[Document(page_content='THE PHILOSOPHY \nOF \nTHE TEACHINGS \nOF ISLAM \n \nHadrat Mirza Ghulam Ahmad  \nThe Promised Messiah and Mahdias  \nFounder of the Ahmadiyya Muslim Jam a‘at \n \nTranslated into English by \nSir Muhammad Zafrulla Khan \n \n \nIslam International Publications Ltd.', metadata={'source': 'data/Philosophy-of-Teachings-of-Islam.pdf', 'page': 2}),
 Document(page_content='“The Philosophy of the Teachings of Islam” \nby Hadrat Mirza Ghulam Ahmad as  of Qadian \nEnglish rendering of “ Isl ami Usu l K i Philosophy ” (Urdu) \nBy Sir Muhammad Zafrulla Khan   \nFirst Urdu edition published in 1905, followed by se veral editions.  \nFirst English edition published in UK by  \nThe London Mosque in 1979 \nReprinted  by Islam International Publications Ltd. \nin 1989, 1992, 1996, 2007 \nThe revised edition (in a new format) printed in UK in 201 0 \nReprinted in UK in 2013 & 2017  \n© Islam International Publications Ltd.  \nNo part of this book may be reproduced or used in any 

In [24]:
embeddings = OllamaEmbeddings(model="mxbai-embed-large")

db = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    collection_name="ama-chroma",
    persist_directory="./chroma_db"
    )

In [11]:
# load from disk
db = Chroma(collection_name="ama-chroma", persist_directory="./chroma_db")

In [25]:
db

In [26]:
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnableLambda, RunnablePassthrough


In [49]:
# ## Design ChatPrompt Template
# from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
# prompt = ChatPromptTemplate.from_template(
#     """                                      
#     Answer the following question based only on the provided context. 
#     Think step by step before providing a detailed answer. 
#     I will tip you $1000 if the user finds the answer helpful. 
#     If you don't know the answer, just say "I don't know"
#     <context>
#     {context}
#     </context>
#     Question: {input}
#     """
#     )

In [58]:
retriever = db.as_retriever()

In [59]:
retriever.invoke("Fourth question")


[Document(page_content='FIRST QUESTION \nThe Physical, Moral and Spiritual \n States of Man \nIn the first few pages of this paper I have set forth \ncertain introductory matters which might at first sight \nseem irrelevant, and yet it is necessary to have a clear \nconcept of those matters for the proper appreciation of \nthe reply to the question that has been set out above. \nThree Types of Human Actions \nThe first question relates to the natural and moral \nand spiritual states of man. Be it known that the holy \nWord of God Almighty, the Holy Quran, has indicated \nthree separate sources of these three states. In other \nwords, it has pointed out three springs out of which \nthese respective states flow. \nFirst Source: Nafsi Amm arah,  \nthe Self That Incites to Evil \nThe first spring which is the source of all natural', metadata={'page': 36, 'source': 'data/Philosophy-of-Teachings-of-Islam.pdf'}),
 Document(page_content='FIRST QUESTION \nThe Physical, Moral and Spiritual \n St

In [60]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from IPython.display import Markdown as md
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [70]:
prompt = hub.pull("rlm/rag-prompt")
prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [67]:
# Retrieve and generate using the relevant snippets of the blog
retriever = db.as_retriever()

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [75]:
md(f"{rag_chain.invoke('What are all the questions discussed in this book?')}")

I don't know what all the questions discussed in this book are. The context only shows some section headings and page numbers, but it doesn't list out all the questions.

In [56]:
# Chain
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [66]:
chain.invoke({"question": "Who was the founder of Ahmadiyya Community?"})

KeyError: "Input to ChatPromptTemplate is missing variables {'input'}.  Expected: ['context', 'input'] Received: ['context', 'question']"

In [21]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(llm,prompt)


ValueError: Prompt must accept context as an input variable. Received prompt with input variables: ['entity']

In [19]:
chain.invoke("What is the name of this book?")


ValueError: You must provide an embedding function to compute embeddings.https://docs.trychroma.com/embeddings

In [91]:
from langchain.chains import create_retrieval_chain

retriever = db.as_retriever()
# rchain = create_retrieval_chain(retriever, document_chain)
retriever

VectorStoreRetriever(tags=['Chroma'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7ff08ff89630>)

In [97]:
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)


In [99]:
response=retrieval_chain.invoke({"input":"Chain of thought (CoT; Wei et al. 2022) has become a standard prompting technique"})
response

ValueError: You must provide an embedding function to compute embeddings.https://docs.trychroma.com/embeddings

In [66]:
# query it
query = "Philosphy of teachings of Islam?"
docs = db.similarity_search(query)

# print results
print(docs[0].page_content)

FOURTH QUESTION ....................................... ...... 170  
The Operation of the Practical Ordinances of the Law in this 
Life and the Next ................................................................................. 170  
The Philosophy of Allah’s Swearing by Various things ................. 174  
FIFTH QUESTION ........................................ ......... 182  
Sources of Divine Knowledge ........................................................... 182  
The Nature of Human Conscience ................................................... 187  
Meaning of Revelation ........................................................................ 191  
A Characteristic of Islam .................................................................... 196


In [75]:
from langchain.prompts import ChatPromptTemplate

# prompt = ChatPromptTemplate.from_template(
#     """Answer the following question based on the provided context:
#     <context>
#     {context}
#     </context>
#     Question: {input}"""
# )

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)


In [79]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

setup = RunnableParallel(context=db, question=RunnablePassthrough())
setup.invoke("What is name of this book?")


TypeError: Expected a Runnable, callable or dict.Instead got an unsupported type: <class 'langchain_community.vectorstores.chroma.Chroma'>

In [ ]:
chain = setup | prompt | llm | parser
chain.invoke("What color is Patricia's car?")


In [68]:
from langchain.chains import create_retrieval_chain

retriever = db.as_retriever()
rchain = create_retrieval_chain(retriever, chain)

In [76]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

setup = RunnableParallel(context=retriever, question=RunnablePassthrough())
setup.invoke("What is the name of book? and who wrote it?")


{'context': [Document(page_content='FOURTH QUESTION ....................................... ...... 170  \nThe Operation of the Practical Ordinances of the Law in this \nLife and the Next ................................................................................. 170  \nThe Philosophy of Allah’s Swearing by Various things ................. 174  \nFIFTH QUESTION ........................................ ......... 182  \nSources of Divine Knowledge ........................................................... 182  \nThe Nature of Human Conscience ................................................... 187  \nMeaning of Revelation ........................................................................ 191  \nA Characteristic of Islam .................................................................... 196', metadata={'page': 7, 'source': 'data/Philosophy-of-Teachings-of-Islam.pdf'}),
  Document(page_content='FOURTH QUESTION ....................................... ...... 170  \nThe Operatio

In [14]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import UnstructuredEPubLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate

from langchain.chains import RetrievalQA

In [ ]:
loader = PyPDFDirectoryLoader('./data/')

# OpenAI